In [46]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
import warnings
warnings.simplefilter("ignore")

In [47]:
# --- DATA READ --- #

conn = psycopg2.connect(host='ec2-63-34-177-108.eu-west-1.compute.amazonaws.com', port=5432, dbname='d4ftrbg8effcuj', user='u5vvfpvq86odcq', password='p5864af5d23a46f938318697ef8ee17e16c2a4de63e9198d592e548be83be8320')

sql = """
select houses.name, houses.state, destinations.name ->> 'fr' as destination, 
CASE WHEN houses.created_at::date < '1/1/2020' then '1/1/2020' else houses.created_at::date end as created_at,
'12/31/2023'::date as max_date
from houses
left join destinations on destinations.id = houses.destination_id
where houses.state in ('published', 'hidden', 'warning')
"""

df = sqlio.read_sql_query(sql, conn)

conn = None

In [48]:
# --- DATA TYPE ---#
#convert date columns to datetime
df['created_at'] = pd.to_datetime(df['created_at'])
df['max_date'] = pd.to_datetime(df['max_date'])

In [49]:
# --- DATA TRANSFORMATION --- #
# create a new dataframe that contains one row per day of stay
df = pd.concat([pd.DataFrame({'name': r['name'], 'destination': r['destination'], 'state': r['state'], 'possible_days': pd.date_range(start=r['created_at'], end=r['max_date'], freq='D')}) for i, r in df.iterrows()], ignore_index=True)

period_type = 'M'
# group by period
df['period'] = df['possible_days'].dt.to_period(period_type)
df = df.groupby(['name', 'destination', 'state', 'period']).agg({'possible_days': 'count'}).reset_index()
df

,name,destination,state,period,possible_days
0,Villa Linda,Punta Cana,published,2022-09,23
1,Villa Linda,Punta Cana,published,2022-10,31
2,Villa Linda,Punta Cana,published,2022-11,30
3,Villa Linda,Punta Cana,published,2022-12,31
4,Villa Linda,Punta Cana,published,2023-01,31
...,...,...,...,...,...
86458,Zaira Estate,Paros,published,2023-08,31
86459,Zaira Estate,Paros,published,2023-09,30
86460,Zaira Estate,Paros,published,2023-10,31
86461,Zaira Estate,Paros,published,2023-11,30


In [50]:
# --- PIVOT --- #
pivoted_df = df.pivot(index=['name','destination', 'state'], columns='period', values='possible_days').fillna(0)

# drop the multi index column
pivoted_df = pivoted_df.reset_index()
pivoted_df

period,name,destination,state,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,...,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
0,Villa Linda,Punta Cana,published,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,31.0,30.0,31.0,30.0,31.0,31.0,30.0,31.0,30.0,31.0
1,A Mandria,Région de Porto-Vecchio,published,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,31.0,30.0,31.0,30.0,31.0,31.0,30.0,31.0,30.0,31.0
2,Alpaca,Méribel,published,31.0,29.0,31.0,30.0,31.0,30.0,31.0,...,31.0,30.0,31.0,30.0,31.0,31.0,30.0,31.0,30.0,31.0
3,Alpine Myth,Sankt Anton,published,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,31.0,30.0,31.0,30.0,31.0,31.0,30.0,31.0,30.0,31.0
4,Alykes Estate,Corfou,published,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,31.0,30.0,31.0,30.0,31.0,31.0,30.0,31.0,30.0,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2858,Villa Échappée,Sainte-Maxime,published,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,31.0,30.0,31.0,30.0,31.0,31.0,30.0,31.0,30.0,31.0
2859,Villas Doppia,Presqu'ile Saint-Tropez,hidden,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,31.0,30.0,31.0,30.0,31.0,31.0,30.0,31.0,30.0,31.0
2860,Villas Iberis,Gassin,published,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,31.0,30.0,31.0,30.0,31.0,31.0,30.0,31.0,30.0,31.0
2861,Wood Penthouse,Saint-Moritz,published,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,31.0,30.0,31.0,30.0,31.0,31.0,30.0,31.0,30.0,31.0


In [51]:
# --- UNPIVOT --- #
unpivoted_df = pivoted_df.melt(id_vars=['name', 'destination', 'state'], var_name='period', value_name='possible_days')
unpivoted_df

,name,destination,state,period,possible_days
0,Villa Linda,Punta Cana,published,2020-01,0.0
1,A Mandria,Région de Porto-Vecchio,published,2020-01,0.0
2,Alpaca,Méribel,published,2020-01,31.0
3,Alpine Myth,Sankt Anton,published,2020-01,0.0
4,Alykes Estate,Corfou,published,2020-01,0.0
...,...,...,...,...,...
137419,Villa Échappée,Sainte-Maxime,published,2023-12,31.0
137420,Villas Doppia,Presqu'ile Saint-Tropez,hidden,2023-12,31.0
137421,Villas Iberis,Gassin,published,2023-12,31.0
137422,Wood Penthouse,Saint-Moritz,published,2023-12,31.0


In [52]:
df_hub = pd.read_csv(r"C:\Users\Benjamin\LC-Clean\Business_dashboard_lab\mapping_hub.csv", sep=';')
unpivoted_df = unpivoted_df.merge(df_hub, on='destination', how='left')

In [53]:
unpivoted_df_final = unpivoted_df[["cluster", "destination", "name", "state", "period", "possible_days"]]
unpivoted_df_final

,cluster,destination,name,state,period,possible_days
0,ROW,Punta Cana,Villa Linda,published,2020-01,0.0
1,Corse,Région de Porto-Vecchio,A Mandria,published,2020-01,0.0
2,Alpes LC,Méribel,Alpaca,published,2020-01,31.0
3,Alpes Autrichienne,Sankt Anton,Alpine Myth,published,2020-01,0.0
4,Grèce,Corfou,Alykes Estate,published,2020-01,0.0
...,...,...,...,...,...,...
137419,French Riviera,Sainte-Maxime,Villa Échappée,published,2023-12,31.0
137420,French Riviera,Presqu'ile Saint-Tropez,Villas Doppia,hidden,2023-12,31.0
137421,French Riviera,Gassin,Villas Iberis,published,2023-12,31.0
137422,Alpes LC,Saint-Moritz,Wood Penthouse,published,2023-12,31.0


In [54]:
# --- DATA READ --- #

conn = psycopg2.connect(host='ec2-63-34-177-108.eu-west-1.compute.amazonaws.com', port=5432, dbname='d4ftrbg8effcuj', user='u5vvfpvq86odcq', password='p5864af5d23a46f938318697ef8ee17e16c2a4de63e9198d592e548be83be8320')

sql = """
SELECT houses.name,
case when check_in_date::date < '1/1/2020' then '1/1/2020' else check_in_date::date end as check_in_date,
case when check_out_date::date >= '1/1/2024' then '12/31/2023' else check_out_date::date-1 end as check_out_date
FROM bookings
left join houses on houses.id = bookings.house_id
left join destinations on destinations.id = houses.destination_id
where check_out_date > '1/1/2020' and check_in_date < '1/1/2024' and houses.name != 'House test' and (booker_type = 'Contract') and houses.state in ('published', 'hidden', 'warning')
order by check_in_date asc
"""

df = sqlio.read_sql_query(sql, conn)

conn = None

In [55]:
df

,name,check_in_date,check_out_date
0,Chalet Hadrien,2020-01-01,2020-01-03
1,Chalet Aline,2020-01-01,2020-01-04
2,Villa Toudra,2020-01-01,2020-01-03
3,Chalet Ilulissat,2020-01-01,2020-01-03
4,Villa Cairn,2020-01-01,2020-01-03
...,...,...,...
5187,Penthouse Phoenix 901 Le George,2023-12-23,2023-12-29
5188,Appartement Doron Lodge,2023-12-23,2023-12-29
5189,Villa Avana,2023-12-25,2023-12-31
5190,Villa Alham,2023-12-27,2023-12-31


In [56]:
# --- DATA TYPE ---#
#convert date columns to datetime
df['check_in_date'] = pd.to_datetime(df['check_in_date'])
df['check_out_date'] = pd.to_datetime(df['check_out_date'])

In [57]:
# --- DATA TRANSFORMATION --- #
# create a new dataframe that contains one row per day of stay
df = pd.concat([pd.DataFrame({'name': r['name'], 'booking_days_lc': pd.date_range(start=r['check_in_date'], end=r['check_out_date'], freq='D')}) for i, r in df.iterrows()], ignore_index=True)

period_type = 'M'
# group by period
df['period'] = df['booking_days_lc'].dt.to_period(period_type)
df = df.groupby(['name', 'period']).agg({'booking_days_lc': 'count'}).reset_index()
df

,name,period,booking_days_lc
0,A Mandria,2022-06,4
1,A Mandria,2022-08,11
2,A Mandria,2022-09,16
3,A Mandria,2023-07,14
4,Appartement Alaska,2022-12,7
...,...,...,...
4723,Villa Échappée,2022-08,7
4724,Villa Échappée,2023-07,3
4725,Villa Échappée,2023-08,11
4726,Zaira Estate,2023-06,7


In [58]:
# --- PIVOT --- #
pivoted_df = df.pivot(index=['name'], columns='period', values='booking_days_lc').fillna(0)

# drop the multi index column
pivoted_df = pivoted_df.reset_index()
pivoted_df

period,name,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,...,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
0,A Mandria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0
1,Appartement Alaska,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Appartement Arche Lodge,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Appartement Arly,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Appartement Beaurepaire,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,Villa les Arbousiers,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1232,Villa les Cigales,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
1233,Villa les Roches,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,11.0,0.0,0.0,0.0,0.0
1234,Villa Échappée,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,11.0,0.0,0.0,0.0,0.0


In [59]:
# --- UNPIVOT --- #
unpivoted_df = pivoted_df.melt(id_vars=['name'], var_name='period', value_name='booking_days_lc')
unpivoted_df

,name,period,booking_days_lc
0,A Mandria,2020-01,0.0
1,Appartement Alaska,2020-01,0.0
2,Appartement Arche Lodge,2020-01,0.0
3,Appartement Arly,2020-01,0.0
4,Appartement Beaurepaire,2020-01,0.0
...,...,...,...
59323,Villa les Arbousiers,2023-12,0.0
59324,Villa les Cigales,2023-12,0.0
59325,Villa les Roches,2023-12,0.0
59326,Villa Échappée,2023-12,0.0


In [60]:
unpivoted_df_final = unpivoted_df_final.merge(unpivoted_df, on=['name', 'period'], how='left')

In [61]:
unpivoted_df_final = unpivoted_df_final.fillna(0)

In [62]:
unpivoted_df_final

,cluster,destination,name,state,period,possible_days,booking_days_lc
0,ROW,Punta Cana,Villa Linda,published,2020-01,0.0,0.0
1,Corse,Région de Porto-Vecchio,A Mandria,published,2020-01,0.0,0.0
2,Alpes LC,Méribel,Alpaca,published,2020-01,31.0,0.0
3,Alpes Autrichienne,Sankt Anton,Alpine Myth,published,2020-01,0.0,0.0
4,Grèce,Corfou,Alykes Estate,published,2020-01,0.0,0.0
...,...,...,...,...,...,...,...
137419,French Riviera,Sainte-Maxime,Villa Échappée,published,2023-12,31.0,0.0
137420,French Riviera,Presqu'ile Saint-Tropez,Villas Doppia,hidden,2023-12,31.0,0.0
137421,French Riviera,Gassin,Villas Iberis,published,2023-12,31.0,0.0
137422,Alpes LC,Saint-Moritz,Wood Penthouse,published,2023-12,31.0,0.0


In [63]:
# --- DATA READ --- #

conn = psycopg2.connect(host='ec2-63-34-177-108.eu-west-1.compute.amazonaws.com', port=5432, dbname='d4ftrbg8effcuj', user='u5vvfpvq86odcq', password='p5864af5d23a46f938318697ef8ee17e16c2a4de63e9198d592e548be83be8320')

sql = """
SELECT houses.name,
case when check_in_date::date < '1/1/2020' then '1/1/2020' else check_in_date::date end as check_in_date,
case when check_out_date::date >= '1/1/2024' then '12/31/2023' else check_out_date::date-1 end as check_out_date
FROM bookings
left join houses on houses.id = bookings.house_id
left join destinations on destinations.id = houses.destination_id
where check_out_date > '1/1/2020' and check_in_date < '1/1/2024' and houses.name != 'House test' and (booker_type != 'Contract' or type != 'Bookings::SalesOption') and houses.state in ('published', 'hidden', 'warning')
order by check_in_date asc
"""

df = sqlio.read_sql_query(sql, conn)

conn = None

In [64]:
# --- DATA TYPE ---#
#convert date columns to datetime
df['check_in_date'] = pd.to_datetime(df['check_in_date'])
df['check_out_date'] = pd.to_datetime(df['check_out_date'])

In [65]:
# --- DATA TRANSFORMATION --- #
# create a new dataframe that contains one row per day of stay
df = pd.concat([pd.DataFrame({'name': r['name'], 'booking_days_all': pd.date_range(start=r['check_in_date'], end=r['check_out_date'], freq='D')}) for i, r in df.iterrows()], ignore_index=True)

period_type = 'M'
# group by period
df['period'] = df['booking_days_all'].dt.to_period(period_type)
df = df.groupby(['name', 'period']).agg({'booking_days_all': 'count'}).reset_index()
df

,name,period,booking_days_all
0,Villa Linda,2022-10,4
1,Villa Linda,2022-11,20
2,Villa Linda,2022-12,16
3,Villa Linda,2023-01,17
4,Villa Linda,2023-02,8
...,...,...,...
37560,Villas Iberis,2023-11,30
37561,Villas Iberis,2023-12,30
37562,Zaira Estate,2023-06,7
37563,Zaira Estate,2023-07,24


In [66]:
# --- PIVOT --- #
pivoted_df = df.pivot(index=['name'], columns='period', values='booking_days_all').fillna(0)

# drop the multi index column
pivoted_df = pivoted_df.reset_index()
pivoted_df

period,name,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,...,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
0,Villa Linda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,14.0,0.0,7.0,0.0,0.0,0.0,0.0,8.0,0.0
1,A Mandria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,30.0,25.0,0.0,0.0,0.0,0.0
2,Alpaca,7.0,21.0,31.0,30.0,31.0,6.0,0.0,0.0,4.0,...,31.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Alpine Myth,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Alykes Estate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2685,Villa les Roches,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,10.0,25.0,0.0,0.0,0.0,0.0
2686,Villa Échappée,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,19.0,12.0,25.0,30.0,3.0,31.0,30.0,4.0,0.0,0.0
2687,Villas Doppia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2688,Villas Iberis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,31.0,30.0,0.0,0.0,0.0,14.0,7.0,1.0,30.0,30.0


In [67]:
# --- UNPIVOT --- #
unpivoted_df = pivoted_df.melt(id_vars=['name'], var_name='period', value_name='booking_days_all')
unpivoted_df

,name,period,booking_days_all
0,Villa Linda,2020-01,0.0
1,A Mandria,2020-01,0.0
2,Alpaca,2020-01,7.0
3,Alpine Myth,2020-01,0.0
4,Alykes Estate,2020-01,0.0
...,...,...,...
129115,Villa les Roches,2023-12,0.0
129116,Villa Échappée,2023-12,0.0
129117,Villas Doppia,2023-12,0.0
129118,Villas Iberis,2023-12,30.0


In [68]:
unpivoted_df_final = unpivoted_df_final.merge(unpivoted_df, on=['name', 'period'], how='left')

In [69]:
unpivoted_df_final = unpivoted_df_final.fillna(0)

In [70]:
unpivoted_df_final_group = unpivoted_df_final.groupby(['period']).agg({'possible_days':'sum', 'booking_days_lc':'sum', 'booking_days_all':'sum'}).reset_index()
unpivoted_df_final_group['occupancy_rate_lc'] = unpivoted_df_final_group['booking_days_lc']/unpivoted_df_final_group['possible_days']*100
unpivoted_df_final_group['occupancy_rate_all'] = unpivoted_df_final_group['booking_days_all']/unpivoted_df_final_group['possible_days']*100

In [77]:
unpivoted_df_final_group = unpivoted_df_final_group.round(2)

In [72]:
unpivoted_df_final_group.to_csv('occupancy_rate.csv')

In [73]:
unpivoted_df_final_group.to_clipboard()